<a href="https://colab.research.google.com/github/LindseyRRay/Algos/blob/master/colab/ChatDecisionCurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install funcy pandas numpy matplotlib seaborn isoweek ipython-autotime import_ipynb


from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

from google.colab import drive
drive.mount('/content/drive')

## Navigate to collab notebook and import function
# https://colab.research.google.com/drive/1nta1E1GbvWw0NknSNhpMPrH39af3HMhj
%cd "/content/drive/MyDrive/Colab Notebooks/Cresta"


import json
import funcy
import pandas as pd
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import isoweek

import datetime


#import colab_data_construction_helpers
import colab_config
import colab_data_construction_helpers

%load_ext autotime

%matplotlib inline

!python --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 7.3 MB/s 
Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!
Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Cresta
Python 3.7.13
time: 121 ms (started: 2022-09-09 18:18:42 +00:00)


In [ ]:
from colab_config import (production_files_dir, cleaned_production_filename, analytics_files_dir, clean_analytics_files_dir, 
                          topic_counts_filename, analytics_manager_transfer_dir, merged_files_dir)

time: 703 µs (started: 2022-09-09 18:18:43 +00:00)


In [ ]:
analytics_files = sorted([x for x in os.listdir(merged_files_dir) if x.endswith('.csv') and x.startswith('intuitCS')])
print(analytics_files)

['intuitCS_anaprod_12March2022_1.csv', 'intuitCS_anaprod_12March2022_2.csv', 'intuitCS_anaprod_12March2022_3.csv', 'intuitCS_anaprod_12March2022_4.csv', 'intuitCS_anaprod_12March2022_5.csv', 'intuitCS_anaprod_12March2022_6.csv', 'intuitCS_anaprod_12March2022_7.csv', 'intuitCS_anaprod_12March2022_8.csv']
time: 3.95 s (started: 2022-09-09 18:18:43 +00:00)


In [ ]:
col_subset = ['flag_manager', 'flag_transfer', 'flag_deleted', 'flag_manualclose', 'flag_closeagent', 'universal_agent_username',
    'flag_notpartial', 'chat_month_date','platform_chat_skillname', 'flag_ai_on_mindate', 'chat_duration_minutes',  'flag_manualclose',
    'flag_closeagent',  'flag_notpartial',  'flag_deleted', 
    'flag_resolved',  'number_kb_suggestions',  'total_number_hints', 'n_suggestions_shown', 'disinct_hints',  'receptivity']

time: 869 µs (started: 2022-09-09 18:18:47 +00:00)


In [ ]:
dfs = []
for i in analytics_files[3:8]:
    print(i)
    print(os.path.join(merged_files_dir, i))
    t = pd.read_csv(os.path.join(merged_files_dir, i), usecols=col_subset)
    print('original df shape:', t.shape)
    dfs.append(t)

intuitCS_anaprod_12March2022_4.csv
/content/drive/MyDrive/Research/Cresta/Data/IntData/merged/merged_prod_ana/intuitCS_anaprod_12March2022_4.csv
original df shape: (620292, 17)
intuitCS_anaprod_12March2022_5.csv
/content/drive/MyDrive/Research/Cresta/Data/IntData/merged/merged_prod_ana/intuitCS_anaprod_12March2022_5.csv
original df shape: (583967, 17)
intuitCS_anaprod_12March2022_6.csv
/content/drive/MyDrive/Research/Cresta/Data/IntData/merged/merged_prod_ana/intuitCS_anaprod_12March2022_6.csv
original df shape: (270907, 17)
intuitCS_anaprod_12March2022_7.csv
/content/drive/MyDrive/Research/Cresta/Data/IntData/merged/merged_prod_ana/intuitCS_anaprod_12March2022_7.csv
original df shape: (406824, 17)
intuitCS_anaprod_12March2022_8.csv
/content/drive/MyDrive/Research/Cresta/Data/IntData/merged/merged_prod_ana/intuitCS_anaprod_12March2022_8.csv
original df shape: (99201, 17)
time: 7min 47s (started: 2022-09-09 18:18:47 +00:00)


In [ ]:
df1 = pd.concat(dfs, axis=0, sort=True)

time: 872 ms (started: 2022-09-09 18:31:26 +00:00)


76848
202549
time: 208 ms (started: 2022-09-09 18:31:27 +00:00)


In [ ]:
def compute_skillname_rank(df, output_filename, to_groupby=None, min_chats=0):
    if to_groupby is None:
        to_groupby = ['chat_month_date','platform_chat_skillname']
    print('Grouping by: ', to_groupby)
    analysis_columns = ['chat_duration_minutes', 'flag_manager', 'flag_transfer', 'flag_manualclose', 'flag_closeagent',  'flag_notpartial', 'flag_deleted', 
                        'flag_binary_transfer', 'flag_binary_manager', 'number_kb_suggestions', 'total_number_hints', 'n_suggestions_shown', 'disinct_hints', 'receptivity']
    df.loc[:, analysis_columns] = df[analysis_columns].fillna(0)
    df['flag_binary_manager'] = 0
    mask = (df['flag_manager'] > 0)
    df.loc[mask, 'flag_binary_manager'] = 1
    print(df['flag_binary_manager'].sum())
    df['flag_binary_transfer'] = 0
    mask = (df['flag_transfer'] > 0)
    df.loc[mask, 'flag_binary_transfer'] = 1
    print(df['flag_binary_transfer'].sum())
    
    pre_ai_counts = df.groupby(to_groupby).agg({'platform_chat_skillname': 'count',
                                                                               'chat_duration_minutes': np.mean,
                                                                               'flag_manager': np.mean,
                                                                               'flag_transfer': np.mean,
                                                                               'flag_manualclose': np.mean,
                                                                               'flag_closeagent': np.mean,
                                                                               'flag_notpartial': np.mean,
                                                                               'flag_deleted': np.mean, 
                                                                                'flag_binary_transfer': np.mean,
                                                                               'flag_binary_manager': np.mean,
                                                                                'number_kb_suggestions': np.mean,
                                                                               'total_number_hints': np.mean,
                                                                               'n_suggestions_shown': np.mean,
                                                                               'disinct_hints': np.mean,
                                                                               'receptivity': np.mean})
    
    pre_ai_counts = pre_ai_counts.rename(columns={'platform_chat_skillname':'count_chats'}).reset_index()
    pre_ai_counts['number_months'] = pre_ai_counts.groupby('platform_chat_skillname')['chat_month_date'].transform('count')
    ## drop if number of chats in a given month-category below a certain threshold
    mask = (pre_ai_counts['count_chats']>min_chats)
    print(sum(mask))
    pre_ai_counts2 = pre_ai_counts.loc[mask, :].copy()
    print('Pre filter : ', pre_ai_counts.shape, ' post filter ', pre_ai_counts2.shape)
    pre_ai_counts2['rank_chats'] = pre_ai_counts2.groupby(['chat_month_date'])['count_chats'].rank(ascending=False, method='min')

    #pre_ai_counts2['rank_duration'] = pre_ai_counts2.groupby(['chat_month_date'])['chat_duration_minutes'].rank(ascending=True, method='min')

    ## break thse into buckets by month, quarters
    #df.groupby('country')[['value']].transform(lambda x: pd.cut(x, bins = 2).astype(str)
    #pre_ai_counts2['quantile_chat_ranks'] = pre_ai_counts2.groupby('chat_month_date')['count_chats'].transform(lambda x: pd.qcut(x, q=5, labels=['q0', 'q1', 'q2', 'q3', 'q4']))

    print('Saving to ', output_filename)
    pre_ai_counts2.to_csv(output_filename, index=False)
    return pre_ai_counts2

time: 3.91 ms (started: 2022-09-09 18:32:27 +00:00)


In [ ]:
pre_ai_counts = compute_skillname_rank(df1, '/content/drive/MyDrive/Research/Cresta/Data/IntData/chats/chat_ranks_all.csv',
                                    ['chat_month_date','platform_chat_skillname', 'flag_ai_on_mindate'], min_chats=5)

Grouping by:  ['chat_month_date', 'platform_chat_skillname', 'flag_ai_on_mindate']
76848
202549
1591
Pre filter :  (2211, 19)  post filter  (1591, 19)
Saving to  /content/drive/MyDrive/Research/Cresta/Data/IntData/chats/chat_ranks_all.csv
time: 2.59 s (started: 2022-09-09 18:32:33 +00:00)


In [ ]:
pre_ai_counts = compute_skillname_rank(df1, '/content/drive/MyDrive/Research/Cresta/Data/IntData/chats/chat_ranks_agents.csv',
                                    ['chat_month_date','platform_chat_skillname', 'flag_ai_on_mindate', 'universal_agent_username'], min_chats=0)

Grouping by:  ['chat_month_date', 'platform_chat_skillname', 'flag_ai_on_mindate', 'universal_agent_username']
76848
202549
86075
Pre filter :  (86075, 20)  post filter  (86075, 20)
Saving to  /content/drive/MyDrive/Research/Cresta/Data/IntData/chats/chat_ranks_agents.csv
time: 4.14 s (started: 2022-09-09 18:32:39 +00:00)


## Analysis Using the Production Dataset

In [ ]:
print('Reading in production from :', cleaned_production_filename)
prod_df2 = colab_data_construction_helpers.get_prod_df(create_df=False, cleaned_production_filename=colab_config.cleaned_production_filename)

fill_missing_columns = ['duration_minutes', 'n_messages_visitor', 'n_messages_agent', 'number_messages','n_suggestions_shown',
 'flag_transfer', 'flag_manager',  'flag_resolved', 'number_kb_suggestions', 'receptivity',
 'n_followed_hints',  'scored_hints', 'inactivity_hints',  'guidance_hints',
 'positive_reinforce_hints',  'proactive_hints', 'react_game_hints',
 'flag_receptive_noninactive',  'total_number_hints',
 'disinct_hints',  'flag_agent_duplicated', 'sc_suggestions_used', 'sc_ai_used',
 'sc_coach_used', 'number_sc_inserted',  'number_instantsend_suggestions']

prod_df2.loc[:, fill_missing_columns] = prod_df2.loc[:, fill_missing_columns].fillna(0)

prod_df2['flag_binary_manager'] = 0
mask = (prod_df2['flag_manager'] > 0)
prod_df2.loc[mask, 'flag_binary_manager'] = 1
print(prod_df2['flag_binary_manager'].sum())
prod_df2['flag_binary_transfer'] = 0
mask = (prod_df2['flag_transfer'] > 0)
prod_df2.loc[mask, 'flag_binary_transfer'] = 1
print(prod_df2['flag_binary_transfer'].sum())

prod_df2[['flag_binary_transfer', 'flag_binary_manager']].mean()


post_ai_counts = prod_df2.loc[(prod_df2['flag_ai_on_mindate']>0),:].groupby(['chat_month_date','platform_chat_skill']).agg({'platform_chat_skill': 'count',
                                                                               'duration_minutes': np.mean,
                                                                               'flag_resolved': np.mean,
                                                                               'number_kb_suggestions': np.mean,
                                                                               'total_number_hints': np.mean,
                                                                               'n_suggestions_shown': np.mean,
                                                                               'flag_manager': np.mean,
                                                                               'flag_transfer': np.mean,
                                                                               'disinct_hints': np.mean,
                                                                               'receptivity': np.mean, 
                                                                               'flag_binary_transfer': np.mean,
                                                                               'flag_binary_manager': np.mean,})

post_ai_counts = post_ai_counts.rename(columns={'platform_chat_skill':'count_chats'}).reset_index()
post_ai_counts['number_months'] = post_ai_counts.groupby('platform_chat_skill')['chat_month_date'].transform('count')
## drop if number of chats in a given month-category below a certain threshold
mask = (post_ai_counts['count_chats']>5)
print(sum(mask))
post_ai_counts2 = post_ai_counts.loc[mask, :].copy()
print('Pre filter : ', post_ai_counts.shape, ' post filter ', post_ai_counts2.shape)
post_ai_counts2['rank_chats'] = post_ai_counts2.groupby(['chat_month_date'])['count_chats'].rank(
        ascending=False, method='min')

post_ai_counts2['rank_duration'] = post_ai_counts2.groupby(['chat_month_date'])['duration_minutes'].rank(
        ascending=True, method='min')
post_ai_counts2['rank_resolution'] = post_ai_counts2.groupby(['chat_month_date'])['flag_resolved'].rank(
        ascending=False, method='min')
## break thse into buckets by month, quarters
#df.groupby('country')[['value']].transform(lambda x: pd.cut(x, bins = 2).astype(str)
post_ai_counts2['quantile_chat_ranks'] = post_ai_counts2.groupby('chat_month_date')[
        'count_chats'].transform(lambda x: pd.qcut(x, q=5, labels=['q0', 'q1', 'q2', 'q3', 'q4']))
post_ai_counts2['res_per_hour'] = post_ai_counts2['flag_resolved']/(post_ai_counts2['duration_minutes']/60)

post_ai_counts2.to_csv(colab_config.topic_counts_filename, index=False)